# Load Data
Reads from excel files and exports to invidual, bronze layer tables

### Library Definition and Data Import Model

In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import math
import sqlalchemy
import csv
import gc
import scipy
from config import Config

# custom class for managing data load and export
from CustomLibs.ImportTarget import ImportTarget
from CustomLibs.CustomFunctions import sqlcol


# localdbname='Facilities'
# localservername='localhost'
engine = sqlalchemy.create_engine(Config.CONN_STR)


### Data Load From CSV, export to SQL

In [ ]:
targets=[]

#Conversion lookup to set export dtypes explicitly
type_dict={'i':sqlalchemy.types.INT(),
           'f':sqlalchemy.types.FLOAT(),
           'd':sqlalchemy.types.Date(),
           's10':sqlalchemy.types.VARCHAR(length=10),
           's100':sqlalchemy.types.VARCHAR(length=100)}

# Read import definitions from csv file
with open('.\Raw Data\import_ranges.csv',"r",encoding="utf-8") as filein:
    import_ranges=csv.DictReader(filein)
    for row in import_ranges:
        importflag = int(row.pop('importflag'))
        if importflag == 0:
            print(f'Skipping {row["schema"]}.{row["table"]}')
            continue
        # print(f'{row["schema"]}.{row["table"]}')
        row['types']=[sqlalchemy.types.BIGINT()]+[type_dict[x] for x in row['types'].split(',')]
        targets.append(ImportTarget(row))

# For each import definition, read the data and export to sql
for target in targets:
    target.load_data()
    target.export_data(engine)
    target.df=None

#clean up
del targets
_ = gc.collect()

